In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Lambda
from tensorflow.keras.layers import Conv3D, MaxPooling3D, ZeroPadding3D
from tensorflow.keras.optimizers import Adam

In [14]:
def get_3dConv_model():
    model = Sequential()

    # 1st layer group
    model.add(Conv3D(64, (3, 3, 3), activation="relu",name="conv1", 
                     input_shape=(10,150,150,3),
                     strides=(1, 1, 1), padding="same"))  
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid"))

    # 2nd layer group  
    model.add(Conv3D(128, (3, 3, 3), activation="relu",name="conv2", 
                     strides=(1, 1, 1), padding="same"))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid"))

    # 3rd layer group   
    model.add(Conv3D(256, (3, 3, 3), activation="relu",name="conv3a", 
                     strides=(1, 1, 1), padding="same"))
    model.add(Conv3D(256, (3, 3, 3), activation="relu",name="conv3b", 
                     strides=(1, 1, 1), padding="same"))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool3", padding="valid"))

    # 4th layer group  
    model.add(Conv3D(256, (3, 3, 3), activation="relu",name="conv4a", 
                     strides=(1, 1, 1), padding="same"))   
    model.add(Conv3D(256, (3, 3, 3), activation="relu",name="conv4b", 
                     strides=(1, 1, 1), padding="same"))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool4", padding="valid"))

    # 5th layer group  
    model.add(Conv3D(512, (3, 3, 3), activation="relu",name="conv5a", 
                     strides=(1, 1, 1), padding="same"))   
    model.add(Conv3D(512, (3, 3, 3), activation="relu",name="conv5b",
                      strides=(1, 1, 1), padding="same"))
    model.add(ZeroPadding3D(padding=(0, 1, 1)))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool5", padding="valid"))
    model.add(Flatten())
                     
    # FC layers group
    model.add(Dense(2, activation='softmax', name='fc8'))

    model.compile(
        optimizer=Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [15]:
my_model = get_3dConv_model()
my_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 10, 150, 150, 64)  5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 10, 75, 75, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 10, 75, 75, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 5, 37, 37, 128)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 5, 37, 37, 256)    884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 5, 37, 37, 256)    1769728   
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 5, 18, 18, 256)   

In [16]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [21]:
a = ['frame0.jpg', 'frame1.jpg', 'frame10.jpg', 'frame100.jpg', 'frame101.jpg', 'frame102.jpg', 'frame103.jpg', 'frame104.jpg', 'frame105.jpg', 'frame106.jpg', 'frame107.jpg', 'frame108.jpg', 'frame109.jpg', 'frame11.jpg', 'frame110.jpg', 'frame111.jpg', 'frame112.jpg', 'frame113.jpg', 'frame114.jpg', 'frame115.jpg', 'frame116.jpg', 'frame117.jpg', 'frame118.jpg', 'frame119.jpg', 'frame12.jpg', 'frame120.jpg', 'frame121.jpg', 'frame122.jpg', 'frame13.jpg', 'frame14.jpg', 'frame15.jpg', 'frame16.jpg', 'frame17.jpg', 'frame18.jpg', 'frame19.jpg', 'frame2.jpg', 'frame20.jpg', 'frame21.jpg', 'frame22.jpg', 'frame23.jpg', 'frame24.jpg', 'frame25.jpg', 'frame26.jpg', 'frame27.jpg', 'frame28.jpg', 'frame29.jpg', 'frame3.jpg', 'frame30.jpg', 'frame31.jpg', 'frame32.jpg', 'frame33.jpg', 'frame34.jpg', 'frame35.jpg', 'frame36.jpg', 'frame37.jpg', 'frame38.jpg', 'frame39.jpg', 'frame4.jpg', 'frame40.jpg', 'frame41.jpg', 'frame42.jpg', 'frame43.jpg', 'frame44.jpg', 'frame45.jpg', 'frame46.jpg', 'frame47.jpg', 'frame48.jpg', 'frame49.jpg', 'frame5.jpg', 'frame50.jpg', 'frame51.jpg', 'frame52.jpg', 'frame53.jpg', 'frame54.jpg', 'frame55.jpg', 'frame56.jpg', 'frame57.jpg', 'frame58.jpg', 
'frame59.jpg', 'frame6.jpg', 'frame60.jpg', 'frame61.jpg', 'frame62.jpg', 'frame63.jpg', 'frame64.jpg', 'frame65.jpg', 'frame66.jpg', 'frame67.jpg', 'frame68.jpg', 'frame69.jpg', 'frame7.jpg', 'frame70.jpg', 'frame71.jpg', 'frame72.jpg', 'frame73.jpg', 'frame74.jpg', 'frame75.jpg', 'frame76.jpg', 'frame77.jpg', 'frame78.jpg', 'frame79.jpg', 'frame8.jpg', 'frame80.jpg', 'frame81.jpg', 'frame82.jpg', 'frame83.jpg', 'frame84.jpg', 'frame85.jpg', 'frame86.jpg', 'frame87.jpg', 'frame88.jpg', 'frame89.jpg', 'frame9.jpg', 'frame90.jpg', 'frame91.jpg', 'frame92.jpg', 'frame93.jpg', 'frame94.jpg', 'frame95.jpg', 'frame96.jpg', 'frame97.jpg', 'frame98.jpg', 'frame99.jpg']
a.sort(key=natural_keys)

In [22]:
a

['frame0.jpg',
 'frame1.jpg',
 'frame2.jpg',
 'frame3.jpg',
 'frame4.jpg',
 'frame5.jpg',
 'frame6.jpg',
 'frame7.jpg',
 'frame8.jpg',
 'frame9.jpg',
 'frame10.jpg',
 'frame11.jpg',
 'frame12.jpg',
 'frame13.jpg',
 'frame14.jpg',
 'frame15.jpg',
 'frame16.jpg',
 'frame17.jpg',
 'frame18.jpg',
 'frame19.jpg',
 'frame20.jpg',
 'frame21.jpg',
 'frame22.jpg',
 'frame23.jpg',
 'frame24.jpg',
 'frame25.jpg',
 'frame26.jpg',
 'frame27.jpg',
 'frame28.jpg',
 'frame29.jpg',
 'frame30.jpg',
 'frame31.jpg',
 'frame32.jpg',
 'frame33.jpg',
 'frame34.jpg',
 'frame35.jpg',
 'frame36.jpg',
 'frame37.jpg',
 'frame38.jpg',
 'frame39.jpg',
 'frame40.jpg',
 'frame41.jpg',
 'frame42.jpg',
 'frame43.jpg',
 'frame44.jpg',
 'frame45.jpg',
 'frame46.jpg',
 'frame47.jpg',
 'frame48.jpg',
 'frame49.jpg',
 'frame50.jpg',
 'frame51.jpg',
 'frame52.jpg',
 'frame53.jpg',
 'frame54.jpg',
 'frame55.jpg',
 'frame56.jpg',
 'frame57.jpg',
 'frame58.jpg',
 'frame59.jpg',
 'frame60.jpg',
 'frame61.jpg',
 'frame62.jpg',
 '